In [1]:
from data_reader import Vocabulary, HWDBDatasetHelper, LMDBReader

# your path to data
train_path = r'/DATA/ichuviliaeva/ocr_data/train.lmdb'
test_path = r'/DATA/ichuviliaeva/ocr_data/test.lmdb'
gt_path = './gt.txt'

In [2]:
import cv2
import numpy as np
import torchvision
import wandb

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms

/home/avashchilko/abbyy9sem/course_cvdl/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_reader = LMDBReader(train_path)
train_reader.open()
train_helper = HWDBDatasetHelper(train_reader)

In [4]:
train_helper, val_helper = train_helper.train_val_split()

In [5]:
train_helper.size(), val_helper.size()

(2578433, 644609)

In [6]:
torch.manual_seed(42)

In [7]:
from task2pack.utils.train import show_train_plots, train_with_trainable_loss
from task2pack.utils.data import HWDBDataset 

from task2pack.models.resnet import ResNet12GrayscaleFeatPytorch
from task2pack.models.loss import CenterLoss

In [8]:
"""
model = ResNet34GrayscaleFeat(train_helper.vocabulary.num_classes())
"""
model = ResNet12GrayscaleFeatPytorch(train_helper.vocabulary.num_classes())

In [9]:
model_name = 'ResNet12GrayscaleFeatPytorch'

train_transfroms = nn.Sequential(
    transforms.Resize((128, 128))
)

val_transfroms = nn.Sequential(
    transforms.Resize((128, 128))
)

train_dataloader_config = {
    'batch_size': 512,
    'shuffle': True,
    'drop_last': True,
    'num_workers': 8,
}

test_dataloader_config = {
    'batch_size': 2048,
    'shuffle': False,
    'num_workers': 8,
}

training_config = {
    'lr': 1e-3,
    'epochs': 20,
    'milestones': [40, 50, 75],
    'gamma': 0.7,
    'weight_criterion': 0.3,
    'lr_criterion': 0.5,
}

device = 'cuda:1'
criterion = torch.nn.CrossEntropyLoss()
centerloss = CenterLoss(num_classes=train_helper.vocabulary.num_classes(), feat_dim=512)

wandb.init(
    project='ocr task 2',
    name='{} {} epochs with lr={} no augment'.format(model_name, training_config['epochs'], training_config['lr']),
    config={
        'train_dataloader_config': train_dataloader_config,
        'test_dataloader_config': test_dataloader_config,
        'training_config': training_config,
        'train_transforms': train_transfroms,
        'val_transforms': val_transfroms,

        "architecture": model_name,
        "dataset": "CASIA Offline Chinese Handwriting",
        "criterion": "Cross Entropy Loss + Centerloss",
        "optimizer": "Adam + SGD(Centerloss)",
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vashchilkoav. Use `wandb login --relogin` to force relogin


In [10]:
train_dataset = HWDBDataset(train_helper, transforms=train_transfroms)
val_dataset = HWDBDataset(val_helper, transforms=val_transfroms)

In [11]:
train_losses, test_losses, train_centerloss, test_centerloss, trained_model = train_with_trainable_loss(
    train_dataset=train_dataset,
    test_dataset=val_dataset,
    model=model, 
    criterion=criterion,
    trainable_criterion=centerloss,
    train_dataloader_kwargs=train_dataloader_config,
    test_dataloader_kwargs=test_dataloader_config,
    training_kwargs=training_config,
    device=device,
    wandb_instance=wandb,
    eval_every=2,
)

/home/avashchilko/abbyy10sem/course_ocr/task2/task2pack/models/loss.py:30: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  distmat.addmm_(1, -2, x, self.centers.t())


Initial val: [regular_loss: 0.004350887026771366, trainable_loss: 0.076739758697722, accuracy: 5.7399136530827216e-05]
Epoch 1:
Train loss: [regular: 0.007463973710231441 trainable: 0.15181989397094015]
Epoch 2:
Train loss: [regular: 0.0007991192896107778 trainable: 0.038123349090206315]
Val : [regular_loss: 0.0002540232281633176, trainable_loss: 0.005424670172062339, accuracy: 0.8819858239646049]
Epoch 3:
Train loss: [regular: 0.0005518375115188757 trainable: 0.013712905670878744]
Epoch 4:
Train loss: [regular: 0.0004617943626318037 trainable: 0.006215794948965896]
Val : [regular_loss: 0.00013317624369362047, trainable_loss: 0.0011135250057519197, accuracy: 0.932014601099271]
Epoch 5:
Train loss: [regular: 0.0004017283245075096 trainable: 0.0032570890688544407]
Epoch 6:
Train loss: [regular: 0.0003548520273372514 trainable: 0.0019002064942830832]
Val : [regular_loss: 0.00012160782436326708, trainable_loss: 0.000386351981094622, accuracy: 0.937312386268265]
Epoch 7:
Train loss: [regula

KeyboardInterrupt: 

In [ ]:
show_train_plots(train_losses, ctest_losses, 'ResNet34Grayscale CrossEntropy')
show_train_plots(train_centerloss, test_centerloss, 'ResNet34Grayscale Centerloss')

In [13]:
from course_ocr_t2.evaluate import evaluate
from tqdm import tqdm

test_path = r'/DATA/ichuviliaeva/ocr_data/test.lmdb'
pred_path = './pred.txt'

test_reader = LMDBReader(test_path)
test_reader.open()
test_helper = HWDBDatasetHelper(test_reader, prefix='Test')

test_transforms = nn.Sequential(
    transforms.Resize((128, 128)),
)

test_dataset = HWDBDataset(test_helper, transforms=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False, num_workers=8)

preds = []
trained_model.eval()
with torch.no_grad():
    for X, _ in tqdm(test_loader):
        logits, _ = trained_model(X.to(torch.float32).to(device))
        classes = torch.argmax(logits, dim=1).cpu().numpy()
        preds.extend(classes)
    
with open(pred_path, 'w') as f_pred:
    for idx, pred in enumerate(preds):
        name = test_helper.namelist[idx]
        cls = train_helper.vocabulary.class_by_index(pred)
        print(name, cls, file=f_pred)
        
test_accuracy = evaluate('./gt.txt', './pred.txt')
wandb.run.summary['test_accuracy'] = test_accuracy
wandb.run.summary['test_transforms'] = test_transforms

torch.save(trained_model.state_dict(), './model.pth')
wandb.save('./model.pth')
wandb.save('./pred.txt')

wandb.finish()

100%|█████████████████████████████████████████████████████████████████████████████████| 380/380 [02:54<00:00,  2.17it/s]


NameError: name 'trained_model' is not defined